In [17]:
import numpy as np
import matplotlib as mp

#Creacion de los discos
class Discos:
    name = ""
    radio = 0
    masa = 0
    Vx = 0
    Vy = 0
    
    def __init__(self,thename,elradio,lamasa,veloX,veloY):
        self.name = thename
        self.radio = elradio
        self.masa = lamasa
        self.Vx = veloX
        self.Vy = veloY

def main():
    #disco 1
    nom1 = "disco1"
    disc1 = Discos(nom1,5,10,1,1)
    
    print(disc1.Vx)
    '''#disco 2
    nom2 = "disco2"
    rad2 = 5
    masa2 = 10
    disc2 = Discos(nom2,rad2,masa2)
    
    #disco 3
    nom3 = "disco3"
    rad3 = 5
    masa3 = 10
    disc3 = Discos(nom3,rad3,masa3)
    
    #disco 4
    nom4 = "disco4"
    rad4 = 5
    masa4 = 10
    disc4 = Discos(nom4,rad4,masa4)
    '''    
    
        
main()

1
